In [1]:
from rlcard.envs import make
from rlcard.agents.dmc_agent.trainer import DMCTrainer

print("potato")
env = make("doudizhu")
# Initialize the DMC trainer
trainer = DMCTrainer(
    env,
    cuda='',
    load_model=True,
    xpid="doudizhu",
    savedir='experiments/dmc_result',
    save_interval=10,
    num_actor_devices=1,
    num_actors=5,
    training_device="0",
)

# Train DMC Agents
trainer.start()

potato


Found log directory: experiments/dmc_result\doudizhu
Saving arguments to experiments/dmc_result\doudizhu/meta.json
Path to meta file already exists. Not overriding meta.
Saving messages to experiments/dmc_result\doudizhu/out.log
Path to message file already exists. New data will be appended.
Saving logs data to experiments/dmc_result\doudizhu/logs.csv
Saving logs' fields to experiments/dmc_result\doudizhu/fields.csv
Path to log file already exists. New data will be appended.
[INFO:16912 trainer:265 2022-11-23 17:58:50,523] Resuming preempted job, current stats:
{'mean_episode_return_0': 0.4944831430912018, 'loss_0': 0.19426479935646057, 'mean_episode_return_1': 0.5110607743263245, 'loss_1': 0.23292842507362366, 'mean_episode_return_2': 0.5118946433067322, 'loss_2': 0.22866547107696533}
[INFO:16912 trainer:335 2022-11-23 17:59:08,399] Saving checkpoint to experiments/dmc_result/doudizhu/model.tar
[INFO:16912 trainer:367 2022-11-23 17:59:08,804] After 1052800 frames: @ 0.0 fps Stats:
{'l

In [3]:
import torch

import rlcard
from rlcard.agents import RandomAgent
import os
from rlcard.utils import (
    get_device,
    set_seed,
    tournament,
    reorganize,
    Logger,
    plot_curve,
)

def train(eval):

    # Check whether gpu is available
    device = get_device()

    # Seed numpy, torch, random
    set_seed(42)

    num_episodes = 500000
    num_eval_games = 2000

    # Make the environment with seed
    env = rlcard.make(
        "doudizhu",
        config={
            'seed': 42,
        }
    )

    # Initialize the agent and use random agents as opponents
    from rlcard.agents import DQNAgent
#     agent = DQNAgent(
#         num_actions=env.num_actions,
#         state_shape=env.state_shape[0],
#         mlp_layers=[64,64],
#         device=device,
#     )

    agent = torch.load("experiments/doudizhu_dqn/model8000.pth")

    agents = [agent]
    for _ in range(1, 3):
        agents.append(RandomAgent(num_actions=env.num_actions))
    env.set_agents(agents)

    # Start training
    with Logger("experiments/doudizhu_dqn/") as logger:
        for episode in range(8001, num_episodes):
            # Generate data from the environment
            trajectories, payoffs = env.run(is_training=True)

            # Reorganaize the data to be state, action, reward, next_state, done
            trajectories = reorganize(trajectories, payoffs)

            # Feed transitions into agent memory, and train the agent
            # Here, we assume that DQN always plays the first position
            # and the other players play randomly (if any)
            for ts in trajectories[0]:
                agent.feed(ts)

            # Evaluate the performance. Play with random agents.
            # if eval:
            #     logger.log_performance(
            #         env.timestep,
            #         tournament(
            #             env,
            #             num_eval_games,
            #         )[0]
            #     )

            if episode % 1000 == 0 and episode:
                save_path = os.path.join("experiments/doudizhu_dqn/", f'model{episode}.pth')
                torch.save(agent, save_path)

        # Get the paths
        csv_path, fig_path = logger.csv_path, logger.fig_path

    # Plot the learning curve
    plot_curve(csv_path, fig_path, "dqn")

    # Save model
    save_path = os.path.join("experiments/doudizhu_dqn/", 'model.pth')
    torch.save(agent, save_path)
    print('Model saved in', save_path)
train(True)

--> Running on the GPU

----------------------------------------
  timestep     |  55
  reward       |  0.0195
----------------------------------------

----------------------------------------
  timestep     |  117037
  reward       |  0.0185
----------------------------------------

Logs saved in experiments/doudizhu_dqn/


KeyboardInterrupt: 